In [1]:
# 필요 라이브러리 import

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.chrome.options import Options

import pickle

In [2]:
# selenium 속도향상위해 불필요한 옵션을 사용하지 않게하는 코드

options = Options()
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 
                                                    'plugins' : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications' : 2, 'auto_select_certificate': 2,
                                                    'fullscreen' : 2,
                                                    'mouselock' : 2, 'mixed_script': 2, 
                                                    'media_stream' : 2, 'media_stream_mic' : 2,
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2,
                                                    'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2,
                                                    'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2,
                                                    'durable_storage' : 2}}

options.add_experimental_option('prefs', prefs) 
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions")

In [3]:
import time

# 서비스 대분류별(8개 서비스) 유저명단을 보여주는 페이지에서 유저별 id 파싱
# 각 페이지별 무한 스크롤 javaScript 제어를 위한 셀레니움 기능 사용
def get_user_list(base_url):
    chromedriver = 'C:/selenium/chromedriver.exe' 
    driver = webdriver.Chrome(chromedriver) # 크롬드라이버 작동
    base_url = base_url
    
    driver.get(base_url)
    
    SCROLL_PAUSE_TIME = 10

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while True:
        # Scroll down to bottom                                                      
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)                                                
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height            
        new_height = driver.execute_script("return document.body.scrollHeight")
        i+=1
#         print(i)

        if i == 350:                                                
            break
        last_height = new_height
        
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    a_tags = soup.select('#wrapArticle > div.wrap_article_list.\#keyword_related_contents > ul > li > a.link_post')
    
    save_href = []
    for a_tag in a_tags :
        save_href.append(a_tag['href'])
    
    driver.close()
    
    return save_href

In [4]:
# category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g','',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','글쓰기_코치?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g'////,'멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

In [5]:
# '스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g'

In [7]:
# 디자인_스토리, 멋진_캘라그래피, 지구한바퀴_세계여행

categories = ['멋진_캘리그래피?q=g']
for category in categories:
    each_user_id = []
    each_user_id = get_user_list("https://brunch.co.kr/keyword/"+category)
#     print(len(each_user_id))
    with open(category[:-4]+'_userId.txt','wb') as f:
        pickle.dump(each_user_id,f)